# ASR-MT
## Lucida AI Evaluation

### Goal
Investigate the relationship between the errors of ASR (Automatic Speech Recognition) and the errors of MT (Machine Translation).

### Generate Quries from Original Data

100 sentences.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import re, string
from operator import itemgetter
import matplotlib.pyplot as plt
fig = plt.figure()
plt.rc("font", family="serif")
plt.rc("font", size=15)

In [40]:
# Return the Ensligh text corresponding to the specific sentence.
def get_English_text(id):
    links_with_id = links.loc[links['id'] == id]
    for translation_id in links_with_id['translation_id']:
        data_with_id = data.loc[data['id'] == translation_id]
        for index, row in data_with_id.iterrows():
            if row['lang'] == 'eng':
                return row['text']

# Generate 100 queires from the original data files
#'sentences.csv' and 'links.csv'.
# Format: 'Ensligh text','Chinese text'
# Save the queries to 'text/query.txt'.
# Data source: http://tatoeba.org/eng/downloads.
def generate_100_queires():
    data = pd.read_csv('sentences.csv', \
                       names = ["id", "lang", "text"], delimiter='\t')
    chinese_rows = data.loc[data['lang'] == 'cmn']
    links = pd.read_csv('links.csv', \
                        names = ["id", "translation_id"], delimiter='\t')
    num_queries = 100
    header = ['query', 'chinese']
    query_data = pd.DataFrame(np.zeros((num_queries, len(header))), \
                              columns=header)
    # Randomly select 1000 rows, and pick 100 from them as queries.
    chinese_rows = chinese_rows.ix[np.random.choice(chinese_rows.index, 1000)]
    chinese_rows = chinese_rows.assign(english=np.zeros(chinese_rows.shape[0]))
    count = 0
    for index, row in chinese_rows.iterrows():
        if count >= num_queries:
            break
        english = get_English_text(row['id'])
        # Prefer long sentences without '.', ';', '?', or '!'.
        if english is not None and not '.' in english[0:-1] and \
        not ';' in english[0:-1] and not '?' in english[0:-1] and \
        not '!' in english[0:-1] and \
        len(english.split()) >= 10:
            query_data.ix[count, 'query'] = english
            query_data.ix[count, 'chinese'] = row['text']
            count += 1
    if count < num_queries:
        raise RuntimeError('Too few data!')
    query_data.to_csv('text/query.txt')
    
# generate_100_queires()

In [48]:
# Generate the transcript with all the 100 English sentences in it
# stored in 'speech/transcript.txt'.
def generate_transcript():
    query_data = pd.read_csv('text/query.txt')
    file = open('speech/transcript.txt', 'w')
    for english in query_data['english']:
        file.write(english + '\n')
    print('Use the transcript to generate audio files!')
        
# generate_transcript()